In [11]:
# import splitfolders  # or import split_folders

# # Split dengan ratio.
# # Untuk hanya membagi menjadi set pelatihan dan validasi, setel tuple menjadi `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio("dataset", output="output", seed=1337, ratio=(.8,.2), group_prefix=None)
import cv2
import os
import tensorflow as tf
import keras.applications
from keras.applications import vgg19
from keras.models import sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model


img_row, img_col = 224, 224
model = vgg19.VGG19(weights = 'imagenet',
                    include_top= False,
                    input_shape= (img_row, img_col, 3))

for layer in model.layers:
    layer.trainable = False

model.summary()




Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
def layer_adder(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(512, activation = 'relu')(top_model)
    top_model = Dense(num_classes, activation = 'softmax')(top_model)
    return top_model


In [13]:
num_classes = 3

FC_Head = layer_adder(model, num_classes)

model = Model(inputs = model.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = "output/train"
val_data_dir = "output/val"

train_dategen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=45,
                width_shift_range=0.3,
                height_shift_range=0.3,
                horizontal_flip=True,
                fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 1

train_generator = train_dategen.flow_from_directory(
                train_data_dir,
                seed=42,
                target_size=(img_row, img_col),
                batch_size=batch_size,
                class_mode='categorical')
validation_generator = train_dategen.flow_from_directory(
                val_data_dir,
                seed=42,
                target_size=(img_row, img_col),
                batch_size=batch_size,
                class_mode='categorical')

Found 240 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [15]:
from keras.optimizers import adam_v2
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoin = ModelCheckpoint("face_detector.h15",
                            monitor="val_loss",
                            mode='min',
                            save_best_only=True,
                            verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=3,
                        verbose=1,
                        restore_best_weights=True)
callbacks = [earlystop, checkpoin]

model.compile(loss = 'categorical_crossentropy',
            optimizer='Adam',
            metrics=['accuracy'])
nb_train_samples = 240
nb_val_samples = 60

epochs = 1
batch_size = 1

history = model.fit(
        train_generator,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=validation_generator)


240/240 [==============================] - 808s 3s/step - loss: 0.6962 - accuracy: 0.6916 - val_loss: 0.0016 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00161, saving model to face_detector.h15
INFO:tensorflow:Assets written to: face_detector.h15\assets


In [16]:
model.save("models/face_classifier.h5")